<a href="https://colab.research.google.com/github/repro-school/training-fens/blob/main/Python_data_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuroimaging in python: Nilearn, Nibabel and Nipype.

## Goals

1. Demonstrate data input and output with nibabel.
2. Demonstrate interacting with nifi headers.
3. Demonstrate manipulating image volumes.

Nibabel and Nilearn are two very useful packages for neuroimaging data manipulation. 

[Nibabel](https://nipy.org/nibabel/) is a package designed for interacting with neuroimaging data in common formats.

[Nilearn](https://nilearn.github.io/) is a much broader package that has the functionality to perform machine learning. There are examples of all the wonderful things you can do [here](https://nilearn.github.io/auto_examples/index.html). However, I will not be covering machine learning here. I will only be talking about some of the useful functions that nilearn has for plotting/manipulating data.

## Data.

First, let's download our data. This includes some functional data, an atlas a brain mask and a few other things.

In [ ]:
! pip install nibabel
! pip install nilearn

!pip install --upgrade --no-cache-dir gdown

import gdown

url = 'https://drive.google.com/drive/folders/1_l-SQNRzZkaBhp5d2PJd8eu0MHueYL15?usp=sharing&confirm=t'
output = '/content/example_data'
gdown.download_folder(url=url, output=output, quiet=False)

# Interactive viewers.

Nilearn comes with the capacity to interactively view images. To test that this functionality is working, lets plot the first volume of our functional data.

In [ ]:
funcpath='/content/example_data/filtered_func_data.nii.gz'

In [ ]:
import nilearn
from nilearn import image
from nilearn import plotting
import matplotlib.pyplot as plt
firstim=image.index_img(funcpath, 0)
plotting.view_img(firstim,bg_img=False)

## How do I get nifti data into python?

To do this, we need to import nibabel, and call the 'load' function to load our data into memory.

In [ ]:
import nibabel as nib

In [ ]:
img1=nib.load(funcpath)

In [ ]:
img1

We see that this is a nifti image. Nibabel actually seperates this file into 3 components, *the data*, *the header*, and *the affine*.

Let's start with the data.

To access the data itself, we call the *get_data* function.

In [ ]:
data=img1.get_data()

## How do I access different parts of the data?

It is first important to learn something about the structure of the data. To do this, we can ask how many dimensions it has.

In [ ]:
data.shape

In [ ]:
type(data)

We see that the data has 4 dimensions. It is a functional image. The first 3 are space dimensions (X,Y,Z) and the final one is time. 

Thus our data are 64x64x21 voxels with 122 volumes.

Our data are in a *numpy* array. This is just the standard format in which arrays are stored in python. It is not much different to how data are stored in multidimensional arrays in matlab or R.

Therefore, we can access the first volume of our data as follows.

In [ ]:
firstvol=data[:,:,:,0]
firstvol.shape

Here, the colons denote that we want 'all of' the x,y and z dimensions, but the 0 indicates that we only want the first volume. Note that, in python, indexing starts at 0.

Therefore, if we ask for the 122nd volume, as follows.

In [ ]:
#lastvol=data[:,:,:,122]

We will get an error. Since our indexing starts at 0, the 122nd volume is at index 121

In [ ]:
lastvol=data[:,:,:,121]

In [ ]:
lastvol=data[:,:,:,-1]

This can be a little confusing, since the way python does indexing is different to R, MATLAB etc.

Now let's suppose that we wanted to access one slice of the data in the x dimension, we would do this as follows.

Here for instance, we take slice 40 in the X dimension, from the first volume. 

In [ ]:
myslice=data[40,:,:,0]

In [ ]:
import matplotlib.pyplot as plt

Since a slice is 2 dimensional, we can view it as as we would any other image.

In [ ]:
plt.imshow(myslice)

Often, we may be quite interested in plotting data over time.

Let's say for instance, we wanted to plot data from an individual voxel over the course of the run.

Here, we would populate the first 3 dimensions with an index to indicate the voxel we want, but add a colon in the time dimension to indicate we want the whole timeseries. 

In [ ]:
ts=data[22,12,14,:]

In [ ]:
plt.plot(ts)

## How do I perform arithmetic on volumes?

Often, we may want to perform some arithmetic on functional volumes. For instance, sometimes it is useful to make a mean functional image.

Since our data are in numpy format, we can use numpy functions to manipulate them. Therefore, we can simply ask for the mean, specifying that we want to average over the time dimension.

In [ ]:
import numpy as np
meanvol=np.mean(data,axis=3) # Time is the 3rd dimension
meanvol.shape

## How can I save nifti files?

Nibabel also allows you to save nifit files. For instance, we can save our mean functional volume to a nifti image as follows.

In [ ]:
import os
outpath='/content/example_data/outputs'
os.mkdir(outpath)


new_img = nib.Nifti1Image(meanvol,affine=img1.affine,header=img1.header)
fname=os.path.join(outpath,'meanvol.nii.gz')
nib.save(new_img,fname)

Notice that, in order to save to a nifti file, we also need to specify 'header' and 'affine' information (More on this in a moment).

Now we plot our mean functional image

In [ ]:
display=plotting.plot_anat(fname)
plt.show()

In addition to caculating a mean over time, we may also want to calculate a mean across 2 functional runs. To do this, we would do as follows.

Note that we only have 1 set of data and so here we are just averaging across two of the same run (ordinarily, of course, we would average across two different runs).

In [ ]:
runs = [None]*2 # Make an empty list 
runs[0]=data # Data from the first run
runs[1]=data # Data from the second run
runs=np.array(runs)

In [ ]:
runs.shape

Now we have a new numpy array, with the run number as a new first axis. Therefore, we just have to average across this new axis to average the data across runs. 

In [ ]:
run_average=np.mean(runs,axis=0)

In [ ]:
run_average.shape

## How do I interact with the information stored in the nifti header?

In order to access the nifti header, we do as follows.

In [ ]:
print(img1.header)

Here we see a whole bunch of fields. Here I define a function for getting the most important stuff.

In [ ]:
def getniftibits(file):
    import pandas as pd
    
    nifti = nib.load(file)
    VOXSIZE = nifti.header['pixdim'][1:4]
    SHAPE= (nifti.header['dim'][1:5])
    TR = (nifti.header['pixdim'][4:5])
    VOXFRAME=pd.DataFrame(VOXSIZE)
    VOXFRAME=VOXFRAME.T
    SHAPEFRAME=pd.DataFrame(SHAPE)
    SHAPEFRAME=SHAPEFRAME.T
    VOXFRAME.columns=['VoxsizeX','VoxsizeY','VoxsizeZ']
    SHAPEFRAME.columns=['ShapeX','ShapeY','ShapeZ','Volumes']
    CFRAMEi=pd.concat([VOXFRAME,SHAPEFRAME],axis=1)
    CFRAMEi['TR'] = TR 
    return(CFRAMEi)

In [ ]:
v=getniftibits(funcpath)
v

We can see that the voxel size (mm), dimensions, repetition time (s) and number of volumes are all stored in here. 

The *srow* data describe affine transformations that explain how our voxel data relate to positions relative to the magnet isocentre. You can read more on this [here](https://nipy.org/nibabel/coordinate_systems.html) and [here](https://nilearn.github.io/auto_examples/04_manipulating_images/plot_affine_transformation.html#sphx-glr-auto-examples-04-manipulating-images-plot-affine-transformation-py)

This affine information can also be accessed more directly as follows.

In [ ]:
img1.affine

## How do I resample images?

Let's suppose that we wanted to resample some data.

A possible reason for this is that we have some functional data sampled in MNI 2mm space and we want to know what voxels correspond to regions of interest in an atlas.

Unfortunately, our atlas is in MNI 1mm space. Therefore, it may make sense to downsample the atlas into the same space as our data. 

To do this, we would define our MNI 152 in 2mm space as a template (nilearn actually has its own copy of this).

In [ ]:
from nilearn.datasets import load_mni152_template
template = load_mni152_template()

In [ ]:
template.shape

We would then define our atlas.

In [ ]:
atlaspath='/content/example_data/maxprob_vol_rh.nii.gz'

atlas=nib.load(atlaspath)
atlas.shape

In [ ]:
v=getniftibits(atlaspath)
v

This is in MNI 152 1mm space

We would then use the nilearn utility 'resample_to_img' to sample our atlas into 2mm space.

In [ ]:
resampled_img = image.resample_to_img(atlas, template)

In [ ]:
resampled_img.shape

Our atlas is now in the space that we want. And we can then save it as follows.

In [ ]:
fname=os.path.join(outpath,'atlas_resampled.nii.gz')
nib.save(resampled_img,fname)

In [ ]:
from nilearn.plotting import plot_roi

plot_roi(resampled_img,template, display_mode='z', cut_coords=7)

Here, we plot the regions of interest onto the MNI brain.

## How do I plot statistical results onto the brain?

Remember the motor mapping task we did in FSL? I have imported the t statistics here.

In [ ]:
stat_data='/content/example_data/tstat1.nii.gz'

plotting.plot_stat_map(stat_data,threshold=2,display_mode='x',symmetric_cbar=False,cmap='plasma')

# Exercises

## Exercises: A.

1. using the functional data defined in 'data', plot the timecourses from the following voxels: [24,17,10], [34,18,11].
2. Plot the average of the two timecourses.


In [ ]:
# Put your answer here:

In [ ]:
#@title ↓ --- Reveal solution

ts1=data[24,17,10,:]
ts2=data[34,18,11,:]
mtc=np.mean([ts1,ts2],axis=0)

plt.plot(ts1)
plt.plot(ts2)
plt.plot(mtc)


plt.show()

## Exercises: B.

1. Calcuate the [tsnr](http://www.newbi4fmri.com/mini-tutorial-signal-to-noise#:~:text=For%20each%20voxel%2C%20we%20divide%20the%20mean%20signal%20intensity%20over%20time%20by%20the%20standard%20deviation%20of%20voxel%20time%20course) of the functional volume (that defined in 'data').
2. Save this out to a nifti file.
3. Use [this function](https://nilearn.github.io/dev/modules/generated/nilearn.plotting.plot_img.html#nilearn.plotting.plot_img) to plot this new TSNR image. Set vmin to 0. Set the display mode to x.

In [ ]:
# Put your answer here:

In [ ]:
#@title ↓ --- Reveal solution

tsnr = np.mean(data,axis=3) / np.std(data,axis=3)

new_img = nib.Nifti1Image(tsnr,affine=img1.affine,header=img1.header)
fname=os.path.join(outpath,'tsnr.nii.gz')
nib.save(new_img,fname)
nilearn.plotting.plot_img(fname,bg_img=None,vmin=0,display_mode='x')

## Exercises: C.

1. Create a new array: *zscored_data* that zscores data over time ([look here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html))
2. Plot the same timecourses as in A again.

In [ ]:
# Put your answer here:

In [ ]:
#@title ↓ --- Reveal solution

from scipy import stats

zscored_data=stats.zscore(data,axis=3)

ts1=zscored_data[24,17,10,:]
ts2=zscored_data[34,18,11,:]
mtc=np.mean([ts1,ts2],axis=0)

plt.plot(ts1)
plt.plot(ts2)
plt.plot(mtc)


plt.show()

## Exercises: D.

1. In /content/example_data there is a file called 'bold.nii.gz'
2. What is the voxel size?
3. What was the TR?
4. Print the nifti header of this file.

In [ ]:
#@title ↓ --- Reveal solution

funcpath2='/content/example_data/bold.nii.gz'

v=getniftibits(funcpath2)
print(v)

im2=nib.load(funcpath2)
print(im2.header)


## Python as a 'glue' for fMRI software: Nipype

The simplest way of thinking about nipype is as an interface (or set of interfaces) that allow all of our favourite neuroimaging packages to talk to one another and work effectively together. To get nipype to work on your computer, you should visit their[website](https://nipype.readthedocs.io/en/latest/quickstart.html) and take alook at this great [tutorial](https://miykael.github.io/nipype_tutorial/).

Here cannot install the underlying packages we need (FSL/AFNI) etc in the usual way, so we need to rely on something called [neurodesk](https://www.neurodesk.org/) to create an image of the software packages pre-installed on our system.

In [ ]:
# install CVMFS packages for ubuntu/debian:
!pip install nipype
!apt-get update >> /dev/null
!apt-get install lsb-release >> /dev/null
!wget https://ecsft.cern.ch/dist/cvmfs/cvmfs-release/cvmfs-release-latest_all.deb >> /dev/null
!dpkg -i cvmfs-release-latest_all.deb >> /dev/null
!rm -f cvmfs-release-latest_all.deb
!apt-get update >> /dev/null
!apt-get install cvmfs singularity-container tree >> /dev/null

# this just suppresses a few unessary messages
import os
LD_PRELOAD = os.getenv('LD_PRELOAD')
print(LD_PRELOAD)
os.environ['LD_PRELOAD'] = ''

#this makes the /content directory available to the software containers
SINGULARITY_BINDPATH = os.getenv('SINGULARITY_BINDPATH')
print(SINGULARITY_BINDPATH)
os.environ['SINGULARITY_BINDPATH'] = '/content'


#setup cvmfs
!mkdir -p /etc/cvmfs/keys/ardc.edu.au/
!echo "-----BEGIN PUBLIC KEY-----" | sudo tee /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub
!echo "MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAwUPEmxDp217SAtZxaBep" | sudo tee -a /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub
!echo "Bi2TQcLoh5AJ//HSIz68ypjOGFjwExGlHb95Frhu1SpcH5OASbV+jJ60oEBLi3sD" | sudo tee -a /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub
!echo "qA6rGYt9kVi90lWvEjQnhBkPb0uWcp1gNqQAUocybCzHvoiG3fUzAe259CrK09qR" | sudo tee -a /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub
!echo "pX8sZhgK3eHlfx4ycyMiIQeg66AHlgVCJ2fKa6fl1vnh6adJEPULmn6vZnevvUke" | sudo tee -a /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub
!echo "I6U1VcYTKm5dPMrOlY/fGimKlyWvivzVv1laa5TAR2Dt4CfdQncOz+rkXmWjLjkD" | sudo tee -a /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub
!echo "87WMiTgtKybsmMLb2yCGSgLSArlSWhbMA0MaZSzAwE9PJKCCMvTANo5644zc8jBe" | sudo tee -a /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub
!echo "NQIDAQAB" | sudo tee -a /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub
!echo "-----END PUBLIC KEY-----" | sudo tee -a /etc/cvmfs/keys/ardc.edu.au/neurodesk.ardc.edu.au.pub

!echo "CVMFS_USE_GEOAPI=yes" | sudo tee /etc/cvmfs/config.d/neurodesk.ardc.edu.au.conf
!echo 'CVMFS_SERVER_URL="http://cvmfs.neurodesk.org/cvmfs/@fqrn@"' | sudo tee -a /etc/cvmfs/config.d/neurodesk.ardc.edu.au.conf
!echo 'CVMFS_KEYS_DIR="/etc/cvmfs/keys/ardc.edu.au/"' | sudo tee -a /etc/cvmfs/config.d/neurodesk.ardc.edu.au.conf

!echo "CVMFS_HTTP_PROXY=DIRECT" | sudo tee  /etc/cvmfs/default.local
!echo "CVMFS_QUOTA_LIMIT=5000" | sudo tee -a  /etc/cvmfs/default.local

!cvmfs_config setup
!cvmfs_config chksetup
!cvmfs_config probe neurodesk.ardc.edu.au
!ls /cvmfs/neurodesk.ardc.edu.au/
!cvmfs_config stat -v neurodesk.ardc.edu.au
!cvmfs_talk -i neurodesk.ardc.edu.au host probe
!cvmfs_talk -i neurodesk.ardc.edu.au host info


Now we can access all our container images in the neurodesk project (https://neurodesk.github.io/applications/). This includes FSL/ AFNI etc

In [ ]:
!ls /cvmfs/neurodesk.ardc.edu.au/containers

And we can access any tool we need by talking to this image. Here, FSL:

In [ ]:
!singularity exec /cvmfs/neurodesk.ardc.edu.au/containers/fsl_6.0.5.1_20220120/fsl_6.0.5.1_20220120.simg bet

Here, AFNI:

In [ ]:
!singularity exec /cvmfs/neurodesk.ardc.edu.au/containers/afni_21.2.00_20210714/afni_21.2.00_20210714.simg 3dBlurInMask

## A Minimal example: Slice-timing correction in FSL

Let's suppose that we wanted to perform a fairly common task, like slice-timing in FSL. How would we do this in nipype?

As a first step, we would refer to the relevant nipype documentation, which gives us a brief example of how to do this, and lists a set of inputs and outputs.

https://nipype.readthedocs.io/en/0.12.1/interfaces/generated/nipype.interfaces.fsl.preprocess.html#slicetimer

This reference should be considered your **bible** that you consult for guidance about how to run virtually any process within the nipype framework. There are sub-sections for interfaces for [AFNI](https://nipype.readthedocs.io/en/0.12.1/interfaces/generated/nipype.interfaces.afni.preprocess.html),[ANTS](https://nipype.readthedocs.io/en/0.12.1/interfaces/generated/nipype.interfaces.ants.registration.html) [SPM](https://nipype.readthedocs.io/en/0.12.1/interfaces/generated/nipype.interfaces.spm.preprocess.html)  etc....

We won't copy their example verbatim, and we will include our own example data.

In [ ]:
# First we import the necessary modules. This includes the fsl interface.
from nipype.interfaces import fsl
from nipype.testing import example_data

# Next, we indicate the  specific function we are using from that interface.

st = fsl.SliceTimer()

# We then define inputs to this function, including the directory of the 

st.inputs.in_file='/content/example_data/filtered_func_data.nii.gz'
st.inputs.out_file='filtered_func_data_st.nii.gz'

st.inputs.interleaved = True
st.inputs.time_repetition=int(3)

In [ ]:
st._cmd

Because FSL is installed in a slightly diferent way, we need to modify the name of the command. 

In [ ]:
st._cmd='singularity exec /cvmfs/neurodesk.ardc.edu.au/containers/fsl_6.0.5.1_20220120/fsl_6.0.5.1_20220120.simg slicetimer'

Essentially, what nipype does is take these options, translate them and pass them to the command line. We can query what information is being sent to the command line as follows:

In [ ]:
st.cmdline

We then run this as follows, which will create out slice-timing corrected file at */content/filtered_func_data_st.nii.gz*

In [ ]:
st.run()

# A Minimal example: A Simple Workflow.

The real power of nipype isn't simply calling external functions from within python.

Instead, nipype provides a simple method for connecting these functions together, into something called a **workflow**.

Lets say, for instance, that I want to

1. Perform slice-timing correction with FSL
2. I want to  smooth the data using AFNI.
3. I want to  detrend the data using AFNI

This would be a bit of a pain ordinarily, as we would need to switch between these programs. In nipype this is simplified - as we simply need to define a *workflow* that connects all these functions together.

Below I give an annotated example of this. 



### 1. We define nodes and the inputs to these nodes.

Nodes are essentially self-contained functions, such as the slice timing operation we just performed.

Here I add a set of nodes for performing the various tasks I mentioned above. I have annotated as best as I can below.

In [ ]:
# Import the relevant interfaces we are using.
from nipype.interfaces import afni as afni
from nipype.interfaces import fsl as fsl
import nipype.interfaces.utility as util 
import nipype.pipeline.engine as pe


# Node 1, slicetiming correction using FSL
st=pe.Node(interface=fsl.SliceTimer(),name='slicetime')

# Define the inputs
st.inputs.in_file = '/content/example_data/filtered_func_data.nii.gz'
st.inputs.interleaved = True
st.inputs.time_repetition=3
st.interface._cmd='singularity exec /cvmfs/neurodesk.ardc.edu.au/containers/fsl_6.0.5.1_20220120/fsl_6.0.5.1_20220120.simg slicetimer'
st.inputs.out_file= 'func_st.nii.gz'

# Node 2, simultaneous smoothing and brain extraction using AFNI
bim=pe.Node(interface=afni.BlurInMask(),name='smooth')
# Define the inputs
bim.inputs.mask = '/content/example_data/mask.nii.gz' # I give the path to the brain mask.
bim.inputs.fwhm = 3.0 # I want 3mm smoothing. 
bim.inputs.outputtype= 'NIFTI_GZ'
bim.interface._cmd='singularity exec /cvmfs/neurodesk.ardc.edu.au/containers/afni_21.2.00_20210714/afni_21.2.00_20210714.simg 3dBlurInMask'
bim.inputs.out_file='func_st_blur.nii.gz'

# Node 3, detrending using AFNI.
dt = pe.Node(interface=afni.Detrend(),name='detrend')                     
dt.inputs.args = '-polort 3' # I want to remove polynomials up to an order of 3. 
dt.inputs.outputtype= 'NIFTI_GZ'
dt.interface._cmd='singularity exec /cvmfs/neurodesk.ardc.edu.au/containers/afni_21.2.00_20210714/afni_21.2.00_20210714.simg 3dDetrend'
dt.inputs.out_file='func_st_blur_dt.nii.gz'


### 2. Now we name our workflow and define a directory where the outputs will be stored.

In [ ]:
workflow = pe.Workflow(name='MYWORKFLOW')
workflow.base_dir = 'MYWORKFLOW'
workflow.base_dir

### 3. Now we connect our nodes together

In [ ]:
# We connect the output of the slicetiming node and make this the input to the smoothing node.
workflow.connect(st, 'slice_time_corrected_file', bim, 'in_file')

# We connect the output of the smoothing node, and connnect this to the detrend node.
workflow.connect(bim, 'out_file', dt, 'in_file')

It seems quite hard to understand exactly what is happening here. Fortunately, nipype allows us to view everything as a graph. We can write the graph with the following commands.

In [ ]:
workflow.write_graph(graph2use='exec')

Let's look at the graph. 

In [ ]:
from IPython.display import Image
import os

Image(filename=os.path.join(workflow.base_dir,workflow.name,'graph_detailed.png'))

So, to recap:

1. We first perform slicetiming with fsl,
2. We then smooth with AFNI, using the 3Dblurinmask command.
3. We then detrend the data.

Now we have checked our workflow, all that is left is to..

### 4. Run the workflow.

We will recieve progress messages as the workflow executes. It is worth reading these, just so you have some idea of what is going on under the hood.

In [ ]:
result=workflow.run()

Take a look at the outputs that are stored in /content/MYWORKFLOW. See if you can understand them.